In [5]:
import datetime
import os
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader as DataLoader
from matplotlib import pyplot as plt
from audioDataset import AudioDataset
import basic_model as net0
import MusicRecNet as net1

In [6]:
#set run choices
loss_plot = True
verbose = True
epoch_save = False

# set variables
train_dir = './data/training/clips/'
#test_dir = './data/testing/clips/'
gamma = 0.1
num_epochs = 10
batch_size = 64
learning_rate = 1e-4
weight_decay = 1e-5
rho = 0.9
eps = 1e-06

In [7]:
device = torch.device("cpu")

#need to change based on model name
#this calls the constructor of model class setting the choosen model for the run
################################################################################
#model = net0.GenreClassificationANN()
model = net1.MusicClassNet()

################################################################################
#dataloader
train_dataset = AudioDataset(train_dir)
data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

#model optimizer
optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate, rho=rho, eps=eps, weight_decay=weight_decay )

#model scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

# loss function
loss_function = nn.CrossEntropyLoss()

In [47]:
def train():
    model.train()
    model.to(device)

    epoch_losses = []
    n_images = 800;
    n_batches = int(n_images/batch_size)

    for epoch in range(1, num_epochs+1):
        batch_loss = 0
        print(f'Epoch #{epoch}, Start Time: {datetime.datetime.now()}')

        for melspecs, labels in data_loader:
            audios = melspecs.to(device)
            labels = labels.to(device)
        
            # calculate losses and call call model
            output = model(audios)

            # batch loss
            loss = loss_function(output, labels)
            batch_loss += loss.item()

            # backpropogation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # save epoch loss
        epoch_losses += [sum(batch_loss/n_batches)]
        scheduler.step()

        if verbose:
            print(f'Epoch: #{epoch}, Loss: {epoch_losses[epoch-1]}')

        if epoch_save:
            model_folder_dir = './temp_models'
            if not os.path.isdir( model_folder_dir):
                os.mkdir(model_folder_dir)

             # save temp model
            try:
                temp_model_path = model_folder_dir + '/' + str(datetime.datetime.now()) + '_epoch' + str(epoch) + '.pth'
                torch.save(model.state_dict(), temp_model_path)
                if verbose:
                    print(f'Saved model for epoch {epoch} @{temp_model_path}')
            except:
                print('Epoch model save failed')

    #save final model parameters   
    torch.save(model.state_dict())

    # save final loss plot
    if not os.path.exists("./plots"):
        os.makedirs("./plots")
    generate_loss_plot(epoch_losses, f'./plots/loss_plot_{datetime.datetime.now()}.png', show_plot=loss_plot)

def generate_loss_plot(loss, file_loc, show_plot=False):
    epochs = list(range(1, len(loss)+1))
    plt.plot(epochs, loss)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Epoch vs Loss')
    plt.savefig(file_loc)
    if show_plot:
        plt.show()
    plt.close()

In [48]:
if __name__ == "__main__":
    train()

Epoch #1, Start Time: 2023-11-22 14:23:05.694741


RuntimeError: Couldn't find appropriate backend to handle uri ./data/training/clips/disco.00029.wav and format None.